In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder  # Import your Config class
import requests
import json

# Neo4j connection setup
neo4j_uri = "neo4j://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "password"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def create_node(tx, label, properties):
    tx.run(f"CREATE (n:{label} $props)", props=properties)

def upload_to_neo4j(file):
    df = pd.read_csv(file)
    with driver.session() as session:
        for _, row in df.iterrows():
            session.write_transaction(create_node, "DataNode", dict(row))

def visualize_graph():
    nodes = []
    edges = []
    seen_nodes = set()
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        for record in result:
            node1_id = str(record["n"].id)
            node2_id = str(record["m"].id)
            rel_type = record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    # Use the ConfigBuilder to configure graph settings
    config_builder = ConfigBuilder(nodes=nodes, edges=edges)
    config = config_builder.build()

    return agraph(nodes=nodes, edges=edges, config=config)

# Streamlit interface
st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

st.subheader("Graph Visualization")
graph = visualize_graph()
if graph:
    st.write(graph)  # Display the graph
else:
    st.warning("No data available for visualization.")


Overwriting app.py


In [14]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
import requests
import json

neo4j_uri = "neo4j://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "password"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def visualize_graph():
    nodes = []
    edges = []
    seen_nodes = set()
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        for record in result:
            node1_id = str(record["n"].id)
            node2_id = str(record["m"].id)
            rel_type = record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    config_builder = ConfigBuilder(nodes=nodes, edges=edges)
    config = config_builder.build()

    return agraph(nodes=nodes, edges=edges, config=config)

st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

st.subheader("Graph Visualization")
graph = visualize_graph()
if graph:
    st.write(graph)
else:
    st.warning("No data available for visualization.")


Overwriting app.py


In [16]:
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
import pandas as pd

dataset = pd.read_csv("mcu_box_office.csv")
pipeline = AutoClean(dataset)
txt_file = "cleaned_customers.csv"
pipeline.output.to_csv(txt_file, sep=",", index=True, header=True)
loader = CSVLoader(file_path="cleaned_customers.csv")
docs = loader.load()
print(docs)
documents = [Document(page_content=doc.page_content) for doc in docs]
documents


AutoClean process completed in 0.034084 seconds
Logfile saved to: e:\testtool\autoclean.log
[Document(metadata={'source': 'cleaned_customers.csv', 'row': 0}, page_content=': 0\nmovie_title: Iron Man\nmcu_phase: 1\nrelease_date: 2008-05-02\ntomato_meter: 94\naudience_score: 91\nmovie_duration: 126\nproduction_budget: 186,000,000\nopening_weekend: 102,118,668\ndomestic_box_office: 318,604,126\nworldwide_box_office: 585,171,547\nDay: 2\nMonth: 5\nYear: 2008\nproduction_budget_lab: 9'), Document(metadata={'source': 'cleaned_customers.csv', 'row': 1}, page_content=': 1\nmovie_title: The Incredible Hulk\nmcu_phase: 1\nrelease_date: 2008-06-13\ntomato_meter: 67\naudience_score: 70\nmovie_duration: 112\nproduction_budget: 137,500,000\nopening_weekend: 55,414,050\ndomestic_box_office: 134,806,913\nworldwide_box_office: 265,573,859\nDay: 13\nMonth: 6\nYear: 2008\nproduction_budget_lab: 1'), Document(metadata={'source': 'cleaned_customers.csv', 'row': 2}, page_content=': 2\nmovie_title: Iron Man 

[Document(metadata={}, page_content=': 0\nmovie_title: Iron Man\nmcu_phase: 1\nrelease_date: 2008-05-02\ntomato_meter: 94\naudience_score: 91\nmovie_duration: 126\nproduction_budget: 186,000,000\nopening_weekend: 102,118,668\ndomestic_box_office: 318,604,126\nworldwide_box_office: 585,171,547\nDay: 2\nMonth: 5\nYear: 2008\nproduction_budget_lab: 9'),
 Document(metadata={}, page_content=': 1\nmovie_title: The Incredible Hulk\nmcu_phase: 1\nrelease_date: 2008-06-13\ntomato_meter: 67\naudience_score: 70\nmovie_duration: 112\nproduction_budget: 137,500,000\nopening_weekend: 55,414,050\ndomestic_box_office: 134,806,913\nworldwide_box_office: 265,573,859\nDay: 13\nMonth: 6\nYear: 2008\nproduction_budget_lab: 1'),
 Document(metadata={}, page_content=': 2\nmovie_title: Iron Man 2\nmcu_phase: 1\nrelease_date: 2010-05-07\ntomato_meter: 72\naudience_score: 71\nmovie_duration: 124\nproduction_budget: 170,000,000\nopening_weekend: 128,122,480\ndomestic_box_office: 312,433,331\nworldwide_box_office:

In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
import requests
import json
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))
graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="password")
ollama_url = "https://abaa-34-125-207-21.ngrok-free.app"
llm = Ollama(model="llama3", base_url=ollama_url)

st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

uploaded_file = st.file_uploader("Choose a file", type=['csv'])

if uploaded_file is not None:
    dataset = pd.read_csv(uploaded_file)
    st.subheader("Uploaded file")
    with st.expander("View Uploaded Data"):
        st.dataframe(dataset)
    
    st.subheader("Cleaned and converted data")
    with st.spinner("Cleaning and converting to scheme..."):
        pipeline = AutoClean(dataset)
        txt_file = "cleaned_customers.csv"
        pipeline.output.to_csv(txt_file, sep=",", index=True, header=True)
        loader = CSVLoader(file_path="cleaned_customers.csv")
        docs = loader.load()
        documents = [Document(page_content=doc.page_content) for doc in docs]
        with st.expander("View converted data"):
            st.write(documents)
    
    with st.spinner("Converting to graph document..."):
        llm_transformer = LLMGraphTransformer(llm=llm)
        graph_documents = llm_transformer.convert_to_graph_documents(documents)

        with st.expander("Nodes and Relations"):
            st.subheader("Nodes")
            for node in graph_documents[0].nodes:
                st.write(node)  # Display each node

            st.subheader("Relationships")
            for relation in graph_documents[0].relationships:
                st.write(relation)  # Display each relationship

    graph.add_graph_documents(
        graph_documents,
        baseEntityLabel=True,
        include_source=True
    )
    
    

Overwriting app.py


In [24]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
import requests
import json
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

def visualize_graph():
    nodes = []
    edges = []
    seen_nodes = set()
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        for record in result:
            node1_id = str(record["n"].id)
            node2_id = str(record["m"].id)
            rel_type = record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    config_builder = ConfigBuilder(nodes=nodes, edges=edges)
    config = config_builder.build()

    return agraph(nodes=nodes, edges=edges, config=config)

# Neo4j configuration
graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="password")

# Llama3 Ollama configuration
ollama_url = "https://abaa-34-125-207-21.ngrok-free.app"
llm = Ollama(model="llama3", base_url=ollama_url)

# Streamlit app title
st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

# File upload section
uploaded_file = st.file_uploader("Choose a file", type=['csv'])

if uploaded_file is not None:
    # Load CSV file into a DataFrame
    dataset = pd.read_csv(uploaded_file)
    st.subheader("Uploaded file")
    with st.expander("View Uploaded Data"):
        st.dataframe(dataset)
    
    # Run Cypher query to delete all nodes and relationships in Neo4j
    st.subheader("Deleting existing Neo4j nodes and relationships")
    with st.spinner("Deleting existing graph data..."):
        query = "MATCH (n) DETACH DELETE n"  # Cypher query to delete all nodes and relationships
        try:
            graph.query(query)  # Run the delete query
            st.success("Successfully deleted all existing nodes and relationships in Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while deleting nodes: {e}")
    
    # Data cleaning and conversion
    st.subheader("Cleaned and converted data")
    with st.spinner("Cleaning and converting to schema..."):
        pipeline = AutoClean(dataset)
        txt_file = "cleaned_customers.csv"
        pipeline.output.to_csv(txt_file, sep=",", index=True, header=True)
        
        # Load cleaned data
        loader = CSVLoader(file_path="cleaned_customers.csv")
        docs = loader.load()
        documents = [Document(page_content=doc.page_content) for doc in docs]
        with st.expander("View converted data"):
            st.write(documents)
    
    # Convert cleaned data into graph documents
    st.subheader("Graph Conversion")
    with st.spinner("Converting to graph document..."):
        llm_transformer = LLMGraphTransformer(llm=llm)
        graph_documents = llm_transformer.convert_to_graph_documents(documents)

        # Display nodes and relationships
        with st.expander("Nodes and Relations"):
            st.subheader("Nodes")
            for node in graph_documents[0].nodes:
                st.write(node)  # Display each node

            st.subheader("Relationships")
            for relation in graph_documents[0].relationships:
                st.write(relation)  # Display each relationship

    # Add graph documents to Neo4j
    st.subheader("Adding new graph documents to Neo4j")
    with st.spinner("Adding data to Neo4j..."):
        try:
            graph.add_graph_documents(
                graph_documents,
                baseEntityLabel=True,
                include_source=True
            )
            st.success("Successfully added new graph documents to Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while adding graph documents: {e}")

    if graph:
        st.write(graph)
    else:
        st.warning("No data available for visualization.")


Overwriting app.py


In [28]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
import requests
import json
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_community.graphs import Neo4jGraph

# Neo4j driver configuration
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

# Visualization function
def visualize_graph():
    nodes = []
    edges = []
    seen_nodes = set()
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        for record in result:
            node1_id = str(record["n"].id)
            node2_id = str(record["m"].id)
            rel_type = record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    config_builder = ConfigBuilder(nodes=nodes, edges=edges)
    config = config_builder.build()

    return agraph(nodes=nodes, edges=edges, config=config)

# Llama3 Ollama configuration
ollama_url = "https://abaa-34-125-207-21.ngrok-free.app"
llm = Ollama(model="llama3", base_url=ollama_url)

# Streamlit app title
st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

# File upload section
uploaded_file = st.file_uploader("Choose a file", type=['csv'])

if uploaded_file is not None:
    # Load CSV file into a DataFrame
    dataset = pd.read_csv(uploaded_file)
    st.subheader("Uploaded file")
    with st.expander("View Uploaded Data"):
        st.dataframe(dataset.head(5))
    
    # Run Cypher query to delete all nodes and relationships in Neo4j
    st.subheader("Deleting existing Neo4j nodes and relationships")
    with st.spinner("Deleting existing graph data..."):
        query = "MATCH (n) DETACH DELETE n"  # Cypher query to delete all nodes and relationships
        try:
            with driver.session() as session:
                session.run(query)  # Run the delete query
            st.success("Successfully deleted all existing nodes and relationships in Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while deleting nodes: {e}")
    
    # Data cleaning and conversion
    st.subheader("Cleaned and converted data")
    with st.spinner("Cleaning and converting to schema..."):
        pipeline = AutoClean(dataset)
        txt_file = "cleaned_customers.csv"
        pipeline.output.to_csv(txt_file, sep=",", index=True, header=True)
        
        # Load cleaned data
        loader = CSVLoader(file_path="cleaned_customers.csv")
        docs = loader.load()
        documents = [Document(page_content=doc.page_content) for doc in docs]
        with st.expander("View converted data"):
            st.write(documents)
    
    # Convert cleaned data into graph documents
    st.subheader("Graph Conversion")
    with st.spinner("Converting to graph document..."):
        llm_transformer = LLMGraphTransformer(llm=llm)
        graph_documents = llm_transformer.convert_to_graph_documents(documents)

        # Display nodes and relationships
        with st.expander("Nodes and Relations"):
            st.subheader("Nodes")
            for node in graph_documents[0].nodes:
                st.write(node)  # Display each node

            st.subheader("Relationships")
            for relation in graph_documents[0].relationships:
                st.write(relation)  # Display each relationship

    # Add graph documents to Neo4j
    st.subheader("Adding new graph documents to Neo4j")
    with st.spinner("Adding data to Neo4j..."):
        try:
            graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="password")
            graph.add_graph_documents(
                graph_documents,
                baseEntityLabel=True,
                include_source=True
            )
            st.success("Successfully added new graph documents to Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while adding graph documents: {e}")

    # Visualize the graph if data exists
    st.subheader("Graph Visualization")
    graph_visualization = visualize_graph()
    if graph_visualization:
        st.write(graph_visualization)
    else:
        st.warning("No data available for visualization.")


Overwriting app.py


In [44]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_community.graphs import Neo4jGraph

# Neo4j driver configuration
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

def visualize_graph():
    nodes, edges = [], []
    seen_nodes = set()

    with driver.session() as session:
        for record in session.run("MATCH (n)-[r]->(m) RETURN n, r, m"):
            node1_id, node2_id, rel_type = str(record["n"].id), str(record["m"].id), record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    config = ConfigBuilder(nodes=nodes, edges=edges).build()
    return agraph(nodes=nodes, edges=edges, config=config)

def delete_existing_graph_data():
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")

def clean_and_convert_data(dataset):
    pipeline = AutoClean(dataset)
    cleaned_file_path = "cleaned_customers.csv"
    pipeline.output.to_csv(cleaned_file_path, sep=",", index=True, header=True)
    return cleaned_file_path

def load_cleaned_data(file_path):
    loader = CSVLoader(file_path=file_path)
    return [Document(page_content=doc.page_content) for doc in loader.load()]

def add_graph_documents_to_neo4j(graph_documents):
    graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="password")
    graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)

# Streamlit app title
st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

# File upload section
uploaded_file = st.file_uploader("Choose a file", type=['csv'])

if uploaded_file is not None:
    dataset = pd.read_csv(uploaded_file)
    st.subheader("Uploaded file")
    with st.expander("View Uploaded Data"):
        st.dataframe(dataset.head(5))
    
    # Delete existing Neo4j nodes and relationships
    st.subheader("Deleting existing Neo4j nodes and relationships")
    with st.spinner("Deleting existing graph data..."):
        try:
            delete_existing_graph_data()
            st.success("Successfully deleted all existing nodes and relationships in Neo4j.")
        except Exception as e:
            st.error(f"Error deleting nodes: {e}")

    if 'graph_documents' not in st.session_state:
        st.session_state.graph_documents = None

    st.subheader("Graph Conversion")
    with st.spinner("Converting to graph document..."):
        if st.session_state.graph_documents is None:  # Only convert if not already done
            cleaned_file_path = clean_and_convert_data(dataset)
            documents = load_cleaned_data(cleaned_file_path)  # Ensure documents are loaded properly
            llm = Ollama(model="llama3", base_url="https://f692-34-16-170-184.ngrok-free.app")
            llm_transformer = LLMGraphTransformer(llm=llm)
            st.session_state.graph_documents = llm_transformer.convert_to_graph_documents(documents)
        graph_documents = st.session_state.graph_documents

    # Display nodes and relationships
    if graph_documents:
        with st.expander("Nodes and Relations"):
            st.subheader("Nodes")
            for node in graph_documents[0].nodes:
                st.write(node)  # Display each node

            st.subheader("Relationships")
            for relation in graph_documents[0].relationships:
                st.write(relation) 

    # Add graph documents to Neo4j
    st.subheader("Adding new graph documents to Neo4j")
    with st.spinner("Adding data to Neo4j..."):
        try:
            add_graph_documents_to_neo4j(graph_documents)
            st.success("Successfully added new graph documents to Neo4j.")
        except Exception as e:
            st.error(f"Error adding graph documents: {e}")

    # Visualize the graph if data exists
    st.subheader("Graph Visualization")
    graph_visualization = visualize_graph()
    if graph_visualization:
        st.write(graph_visualization)
    else:
        st.markdown("[Go to Neo4j](http://localhost:7474/)")


Overwriting app.py


In [37]:
%%writefile app.py
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from streamlit_agraph import agraph, Config, Node, Edge
from config import ConfigBuilder
import requests
import json
from AutoClean import AutoClean
from langchain.schema import Document
from langchain.document_loaders import CSVLoader 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_community.graphs import Neo4jGraph

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

def visualize_graph():
    nodes = []
    edges = []
    seen_nodes = set()
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        for record in result:
            node1_id = str(record["n"].id)
            node2_id = str(record["m"].id)
            rel_type = record["r"].type

            if node1_id not in seen_nodes:
                nodes.append(Node(id=node1_id, label=record["n"].get("name", node1_id)))
                seen_nodes.add(node1_id)
            if node2_id not in seen_nodes:
                nodes.append(Node(id=node2_id, label=record["m"].get("name", node2_id)))
                seen_nodes.add(node2_id)

            edges.append(Edge(source=node1_id, target=node2_id, label=rel_type))

    if not nodes:
        return None

    config_builder = ConfigBuilder(nodes=nodes, edges=edges)
    config = config_builder.build()

    return agraph(nodes=nodes, edges=edges, config=config)

ollama_url = "https://abaa-34-125-207-21.ngrok-free.app"
llm = Ollama(model="llama3", base_url=ollama_url)

st.title("RAG Graph with Neo4j, Llama3 (Ollama), and CSV Upload")

uploaded_file = st.file_uploader("Choose a file", type=['csv'])

if uploaded_file is not None:
    dataset = pd.read_csv(uploaded_file)
    st.subheader("Uploaded file")
    with st.expander("View Uploaded Data"):
        st.dataframe(dataset)

    st.subheader("Deleting existing Neo4j nodes and relationships")
    with st.spinner("Deleting existing graph data..."):
        query = "MATCH (n) DETACH DELETE n"
        try:
            with driver.session() as session:
                session.run(query)
            st.success("Successfully deleted all existing nodes and relationships in Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while deleting nodes: {e}")

    st.subheader("Cleaned and converted data")
    with st.spinner("Cleaning and converting to schema..."):
        pipeline = AutoClean(dataset)
        txt_file = "cleaned_customers.csv"
        pipeline.output.to_csv(txt_file, sep=",", index=True, header=True)
        
        loader = CSVLoader(file_path="cleaned_customers.csv")
        docs = loader.load()
        documents = [Document(page_content=doc.page_content) for doc in docs]
        
        st.write("First few converted documents:")
        for doc in documents[:10]:
            st.write(doc.page_content)

    if 'graph_documents' not in st.session_state:
        st.session_state.graph_documents = None

    st.subheader("Graph Conversion")
    with st.spinner("Converting to graph document..."):
        if st.session_state.graph_documents is None:
            llm_transformer = LLMGraphTransformer(llm=llm)
            st.session_state.graph_documents = llm_transformer.convert_to_graph_documents(documents)
        graph_documents = st.session_state.graph_documents

    if graph_documents:
        with st.expander("Nodes and Relations"):
            st.subheader("Nodes")
            for node in graph_documents[0].nodes:
                st.write(node)

            st.subheader("Relationships")
            for relation in graph_documents[0].relationships:
                st.write(relation)

    st.subheader("Adding new graph documents to Neo4j")
    with st.spinner("Adding data to Neo4j..."):
        try:
            graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="password")
            graph.add_graph_documents(
                graph_documents,
                baseEntityLabel=True,
                include_source=True
            )
            st.success("Successfully added new graph documents to Neo4j.")
        except Exception as e:
            st.error(f"An error occurred while adding graph documents: {e}")

    st.subheader("Graph Visualization")
    graph_visualization = visualize_graph()
    if graph_visualization:
        st.write(graph_visualization)
    else:
        st.warning("No data available for visualization.")


Overwriting app.py
